In [1]:
# interactive reimport
%load_ext autoreload
%autoreload 2

In [2]:
import sys, logging, time
import os.path as osp
import torch
from model import Model
from torch_geometric.loader import NeighborLoader
import numpy as np

In [3]:
import argparse
parser = argparse.ArgumentParser(description = 'pytorch version of GraphSAGE')
parser.add_argument('--data', type = str, default = 'cora')
# parser.add_argument('--aggr_func', type = str, default = 'MEAN') # dead argmument
parser.add_argument('--num_epochs', type = int, default = 10)
parser.add_argument('--batch_size', type = int, default = 128)
parser.add_argument('--seed', type = int, default = 13)
parser.add_argument('--cuda', action = 'store_true', help = 'use CUDA')
parser.add_argument('--num_neg_samples', type = int, default = 10) # dead argument
parser.add_argument('--lr', type = float, default = 0.1)
args = parser.parse_args(args=['--cuda'])

In [4]:
args

Namespace(data='cora', num_epochs=10, batch_size=128, seed=13, cuda=True, num_neg_samples=10, lr=0.1)

In [5]:
np.random.seed(args.seed)
torch.manual_seed(args.seed)

logging.basicConfig(level = logging.INFO, format = '%(asctime)s - %(levelname)s - %(mesmodel)s')
args.device = torch.device("cuda" if args.cuda else "cpu")
logging.info('Device:' + str(args.device))

2023-01-14 09:36:48,358 - INFO - Device:cuda


In [32]:
data_name = 'cora' 
attributes_file_name = osp.join('../data', data_name, 'attributes')
labels_file_name = osp.join('../data', data_name, 'labels')
valid_file_name = osp.join('../data', data_name, 'valid_nodes')

features = np.loadtxt(attributes_file_name, dtype=np.float32)
labels = np.loadtxt(labels_file_name, dtype=np.int64)[:,1]
valid_all_nodes_list = np.loadtxt(valid_file_name, dtype = np.int64)

In [34]:
from data_handler import update_viewed_all_nodes_and_edges, generate_whole_graph
def load_graph(t=14):
	stream_edges_dir_name = osp.join('../data', data_name, 'stream_edges')
	viewed_all_nodes, viewed_all_edges = None, None
	for tt in range(t):
		coming_edges = np.loadtxt(osp.join(stream_edges_dir_name, str(tt)), dtype=int)
		viewed_all_nodes, viewed_all_edges = update_viewed_all_nodes_and_edges(
								coming_edges, viewed_all_nodes, viewed_all_edges) 
		graph, valid_nodes = generate_whole_graph(viewed_all_nodes, viewed_all_edges, valid_all_nodes_list, features, labels)
	return graph, valid_nodes

graph, valid_nodes = load_graph()

In [35]:
# Model parameter
input_dim = graph.x.shape[1] # 1433
hidden_dim = 64
output_dim = len(np.unique(graph.y)) # 7
num_layers = 3

In [52]:
# Model definition
model = Model(in_channels=input_dim, hidden_channels=hidden_dim, out_channels=output_dim, num_layers=num_layers).to(args.device)
print(model)
# Model optimizer, may change into adam
optimizer = torch.optim.SGD(model.parameters(), lr = args.lr)

Model(
  (sage): GraphSAGE(1433, 64, num_layers=2)
  (lin): Linear(in_features=64, out_features=7, bias=True)
)


In [37]:
train_mask = np.ones(len(graph.x), dtype=int)
test_mask = np.zeros(len(graph.x), dtype=int)
test_mask[valid_nodes] = 1
train_mask -= test_mask

In [38]:
train_mask.sum(), test_mask.sum()

(1908, 800)

In [39]:
graph.train_mask = train_mask
graph.test_mask = test_mask

In [40]:
graph

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], num_nodes=2708, train_mask=[2708], test_mask=[2708])

In [41]:
from torch_geometric.loader import NeighborLoader

In [42]:
train_loader = NeighborLoader(
	graph, 
	num_neighbors=[args.num_neg_samples] * (num_layers - 1),
	input_nodes=graph.train_mask,
	shuffle=True,
	batch_size=args.batch_size)

In [43]:
sampled_data = next(iter(train_loader))
print(sampled_data)

Data(x=[158, 1433], edge_index=[2, 312], y=[158], num_nodes=158, train_mask=[158], test_mask=[158], input_id=[128], batch_size=128)


In [44]:
sampled_data.test_mask

tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
        1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0,
        0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1,
        0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1,
        0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1], dtype=torch.int32)

In [45]:
sampled_data.train_mask

tensor([0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1,
        0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1,
        1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0,
        1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1,
        1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0,
        1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0], dtype=torch.int32)

In [47]:
sampled_data.x.dtype

torch.float32

In [46]:
next(iter(model.parameters())).dtype

torch.float32

In [48]:
# Model training
from tqdm import tqdm
def train(epoch):
	model.train()
	pbar = tqdm(total=int(len(train_loader.dataset)))
	pbar.set_description(f'Epoch{ epoch:02d}')
	total_loss = 0
	total_examples = 0
	for batch in train_loader:
		batch = batch.to(args.device)
		optimizer.zero_grad()
		loss = model.loss(batch)
		loss.backward()
		optimizer.step()
		loss = loss.data.item()
		total_loss += loss * batch.batch_size
		total_examples += batch.batch_size
		# logging.debug(str(loss.data.item()))
		# if (np.isnan(loss)):
		# 		logging.error('Loss is NaN !!!')
		# 		sys.exit()
		
		# if epoch % 10 == 0:
		# 		logging.info('Epoch: ' + str(epoch) + ' ' + str(np.round(losses / data.train_size, 6)))
		pbar.update(batch.batch_size)
	pbar.close()
	return total_loss/total_examples

In [54]:
for epoch in range(1,11):
    avg_loss = train(epoch)
    print(f'Epoch {epoch:02d}, Loss: {avg_loss:.4f}')
    # train_acc, val_acc, test_acc = test()
    # print(f'Epoch: {epoch:02d}, Train: {train_acc:.4f}, Val: {val_acc:.4f}, '
    #       f'Test: {test_acc:.4f}')

Epoch01: 100%|██████████| 2708/2708 [00:00<00:00, 11572.11it/s]


Epoch 01, Loss: 0.0188


Epoch02: 100%|██████████| 2708/2708 [00:00<00:00, 17934.03it/s]


Epoch 02, Loss: 0.0057


Epoch03: 100%|██████████| 2708/2708 [00:00<00:00, 10109.00it/s]


...

Epoch 08, Loss: 0.0009


Epoch09: 100%|██████████| 2708/2708 [00:00<00:00, 16446.13it/s]


Epoch 09, Loss: 0.0008


Epoch10: 100%|██████████| 2708/2708 [00:00<00:00, 15910.30it/s]

Epoch 10, Loss: 0.0007


In [ ]:
@torch.no_grad()
def test():
	model.eval()
	val_output = model.forward(data.valid_nodes)
	logging.info("Validation Macro F1:" +  str(np.round(f1_score(data.labels[data.valid_nodes], val_output.data.cpu().numpy().argmax(axis=1), average="macro"), 6)))
	logging.info("Validation Micro F1:" +  str(np.round(f1_score(data.labels[data.valid_nodes], val_output.data.cpu().numpy().argmax(axis=1), average="micro"), 6)))
	logging.info("Average batch time:" + str(np.round(np.mean(times), 6)))